In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from crewai import Agent, Task, Crew , LLM

# Let's load environment variables.
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# verification of API's

if not GROQ_API_KEY or not SERPER_API_KEY :
    raise ValueError ("Set necessary API keys in env file.")



os - in order to access env variables and system config
dotenv - for loading API_keys

FAISS - Stores and searches embeddings in order to retrieve the most relevant text chunks
PyPDFLoader - Read's PDF's and convert them into text
RecursiveCharacterTextSplitter - Split long documents into smaller, chunks holding meaning

HuggingFaceEmbeddings - convert text chunks into numberical embeddings (vectors)
chatGroq - Connects langchain to Groq-hosted LLM's for faster inference.

SerperDevTool - For search in internet.
ScrapeWebsiteTool - Extract raw content from webpages.

Agent - helps to define an AI role with goals,tools and behaviour
Task - Describe a specific job an agent must complete
Crew - Orchestrate multiple agents to work together on tasks


In [ ]:
#Initializing main LLM - for routing and final answer.

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=500,
    groq_api_key=GROQ_API_KEY,
    max_retries=2,
)

#Intialize LLM for CrewAI agents

crew_llm_model = LLM(
    model="llama-3.3-70b-versatile",
    temperature=0.7,
    max_tokens=500,
    api_key=GROQ_API_KEY
)


We use the main LLM for routing decisions,final asnwers, and grounding on retireved context - so we keep temperature = 0

so it can be - consistent,repeatable and predictable.

We use crew_llm_model for search query formulation,Summarization,exploration, taking decision on how to approach a task - so we increase the temperature

so it can be - flexible,mild creative, ambigue

